<a href="https://colab.research.google.com/github/movie-genre-team/movie_llm_team/blob/main/backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datasets
datasets.disable_progress_bar()

## Coleta de dados
É feita a coleta de 1000 filmes com gêneros específicos do TMDB. O código atual busca um número fixo de páginas. Preciso modificar o loop de busca para continuar até que 1000 filmes com os gêneros alvo sejam coletados, adicionando também uma salvaguarda contra loops infinitos.

In [ ]:
import requests
import pandas as pd

API_KEY = 'de5194765649190ff5242383212aebe3'
BASE_URL = "https://api.themoviedb.org/3"

target_genres = ["Ação", "Comédia", "Drama", "Ficção científica", "Terror"]
genre_limit = 200
genre_count = {genre: 0 for genre in target_genres}

def get_movies(page=1, language="pt-BR"):
    url = f"{BASE_URL}/movie/popular?api_key={API_KEY}&language={language}&page={page}"
    response = requests.get(url)
    return response.json()

def get_movie_details(movie_id, language="pt-BR"):
    url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}&language={language}"
    response = requests.get(url)
    return response.json()

movies = []
page = 1
max_pages = 200  # Segurança

while not all(count >= genre_limit for count in genre_count.values()) and page <= max_pages:
    data = get_movies(page)
    if not data or not data.get("results"):
        break

    for m in data["results"]:
        details = get_movie_details(m["id"])
        genres = [g["name"] for g in details.get("genres", []) if g["name"] in target_genres]

        assigned_genre = None
        for g in genres:
            if genre_count[g] < genre_limit:
                assigned_genre = g
                break

        if assigned_genre:
            movies.append({
                "id": m["id"],
                "title": m["title"],
                "overview": m["overview"],
                "genre": assigned_genre
            })
            genre_count[assigned_genre] += 1


            if genre_count[assigned_genre] == genre_limit:
                print(f"✅ Coletados {genre_limit} filmes do gênero: {assigned_genre}")

        if all(count >= genre_limit for count in genre_count.values()):
            break

    page += 1

df_balanced = pd.DataFrame(movies)
display(df_balanced.sample(10))
print("Quantidade por gênero:")
print(df_balanced['genre'].value_counts())
print(f"Total de filmes coletados: {len(df_balanced)}")


✅ Coletados 200 filmes do gênero: Drama
✅ Coletados 200 filmes do gênero: Ação
✅ Coletados 200 filmes do gênero: Comédia
✅ Coletados 200 filmes do gênero: Terror
✅ Coletados 200 filmes do gênero: Ficção científica


,id,title,overview,genre
682,422786,Marsietis,,Ficção científica
219,387333,മാർക്ക് ആന്‍റണി,,Drama
184,62327,Terra Prometida,Drama sobre três amigos e a chegada da vida ad...,Drama
687,157336,Interestelar,As reservas naturais da Terra estão chegando a...,Ficção científica
980,72190,Guerra Mundial Z,Um vírus letal se espalha rapidamente e transf...,Ficção científica
963,283995,Guardiões da Galáxia - Vol. 2,Os Guardiões precisam lutar para manter sua re...,Ficção científica
447,27115,O Morto Ambulante,,Terror
249,183011,Liga da Justiça: Ponto de Ignição,"Barry Allen, o herói Flash, nunca conseguiu es...",Ficção científica
551,11936,Cidade das Mulheres,Um empresário se vê preso em um hotel e ameaça...,Comédia
906,693134,Duna: Parte Dois,A jornada de Paul Atreides continua. Ele está ...,Ficção científica


Quantidade por gênero:
genre
Ação                 200
Ficção científica    200
Drama                200
Terror               200
Comédia              200
Name: count, dtype: int64
Total de filmes coletados: 1000



Coletei com sucesso mais de 1000 filmes com os gêneros alvo. O próximo passo, de acordo com a descrição geral da tarefa, é garantir uma distribuição igual de filmes entre os gêneros alvo. Preciso amostrar os filmes coletados para ter um número igual para cada gênero alvo, até um máximo de 200 por gênero para atingir o objetivo de 1000 filmes no total (200 * 5 gêneros).

In [ ]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)




Coletei e equilibrei com sucesso o conjunto de dados por gênero. O próximo passo é preparar os dados para treinar o modelo, o que envolve codificar os rótulos e dividir os dados em conjuntos de treinamento e teste. Também inicializarei o tokenizador e prepararei os conjuntos de dados para o modelo transformer.


In [ ]:
le = LabelEncoder()
df_balanced["label"] = le.fit_transform(df_balanced["genre"])

# treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_balanced["overview"].tolist(), df_balanced["label"].tolist(), test_size=0.2, random_state=42
)

# tokenizer
checkpoint = "distilbert-base-multilingual-cased"  # suporta português
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset  = Dataset.from_dict({"text": test_texts, "label": test_labels})

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Os dados foram preparados e tokenizados. O próximo passo é inicializar o modelo para classificação de sequências usando o checkpoint pré-treinado e o número de rótulos únicos do conjunto de dados balanceado.

In [ ]:
num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=num_labels
)


O modelo foi inicializado. O próximo passo é treinar o modelo usando os conjuntos de dados de treinamento e teste preparados e os argumentos e métricas de treinamento definidos.

In [ ]:
args = TrainingArguments(
    output_dir="movie-genre-classifier",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # pode aumentar
    logging_dir="./logs",
    load_best_model_at_end=False,
    report_to="none"# para wndb
)

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


O modelo foi treinado com sucesso no conjunto de dados balanceado. A etapa final da subtarefa é exibir a probabilidade de cada gênero para uma determinada sinopse de filme usando o modelo treinado.

In [ ]:
def predict_genre(sinopse):
    device = model.device
    inputs = tokenizer(sinopse, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).detach().cpu().numpy()[0]
    pred_id = probs.argmax()
    return le.inverse_transform([pred_id])[0], {genre: float(p) for genre, p in zip(le.classes_, probs)}

sinopse_teste = "Um grupo de amigos enfrenta um monstro em uma floresta escura."
predicted_genre, probabilities = predict_genre(sinopse_teste)

print(f"Sinopse: {sinopse_teste}")
print(f"Gênero Previsto: {predicted_genre}")
print("Probabilidades por Gênero:")
for genre, prob in sorted(probabilities.items(), key=lambda item: item[1], reverse=True):
    print(f"- {genre}: {prob:.4f}")

## Salvar e carregar dados

Salvar os dados coletados e filtrados em um arquivo CSV e carregá-los de volta em um DataFrame.

In [ ]:
df_balanced.to_csv("balanced_movies.csv", index=False)
print("Balanced DataFrame saved to balanced_movies.csv")

df_balanced_loaded = pd.read_csv("balanced_movies.csv")
print("Balanced DataFrame loaded from balanced_movies.csv")
display(df_balanced_loaded.head())

## Preparar dados para treinamento

### Subtarefa:
Executar novamente as etapas de pré-processamento de dados (codificação de rótulo, divisão de treino-teste, tokenização) no novo conjunto de dados balanceado.


Aplica-se codificação de rótulo, dividir dados, tokenizar e formatar conjuntos de dados para treinamento.

## Retreinar o modelo


Treina o modelo de classificação de sequência no conjunto de dados recém-preparado.


Defini os argumentos de treinamento, a função compute_metrics, instanciar o Trainer e iniciar o processo de treinamento conforme as instruções.

In [ ]:
df_balanced_loaded = df_balanced_loaded.dropna(subset=["overview"])

le = LabelEncoder()
df_balanced_loaded["label"] = le.fit_transform(df_balanced_loaded["genre"])

# treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_balanced_loaded["overview"].tolist(), df_balanced_loaded["label"].tolist(), test_size=0.2, random_state=42
)

# tokenizer
checkpoint = "distilbert-base-multilingual-cased"  # suporta português
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset  = Dataset.from_dict({"text": test_texts, "label": test_labels})

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


O treinamento do modelo foi concluído com base na saída da etapa anterior. O próximo passo é exibir a probabilidade de cada gênero para uma determinada sinopse de filme, conforme solicitado na tarefa principal.

In [ ]:
def predict_genre(sinopse):
    device = model.device
    inputs = tokenizer(sinopse, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).detach().cpu().numpy()[0]
    pred_id = probs.argmax()
    return le.inverse_transform([pred_id])[0], {genre: float(p) for genre, p in zip(le.classes_, probs)}

sinopse_teste = "Um grupo de amigos enfrenta um monstro em uma floresta escura."
predicted_genre, probabilities = predict_genre(sinopse_teste)

print(f"Sinopse: {sinopse_teste}")
print(f"Gênero Previsto: {predicted_genre}")
print("Probabilidades por Gênero:")
for genre, prob in sorted(probabilities.items(), key=lambda item: item[1], reverse=True):
    print(f"- {genre}: {prob:.4f}")

## Avaliar e testar





Avalia-se o modelo treinado no conjunto de dados de teste e, em seguida, testar a função de previsão com exemplos de sinopses para demonstrar seu desempenho.

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print("Model Evaluation Results:")
print(eval_results)

# Test with example synopses
example_synopses = [
    """Em Homem-Aranha: Sem Volta para Casa, Peter Parker (Tom Holland) precisará lidar com as consequências
     da sua identidade como o herói mais querido do mundo após ter sido revelada pela reportagem do Clarim Diário,
      com uma gravação feita por Mysterio (Jake Gyllenhaal) no filme anterior. Incapaz de separar sua vida
      normal das aventuras de ser um super-herói, além de ter sua reputação arruinada por acharem que foi ele
      quem matou Mysterio e pondo em risco seus entes mais queridos, Parker pede ao Doutor Estranho (Benedict
       Cumberbatch) para que todos esqueçam sua verdadeira identidade. Entretanto, o feitiço não sai como planejado
        e a situação torna-se ainda mais perigosa quando vilões de outras versões de Homem-Aranha de outro
         universos acabam indo para seu mundo. Agora, Peter não só deter vilões de suas outras versões e fazer
         com que eles voltem para seu universo original, mas também aprender que, com grandes poderes vem grandes responsabilidades.""", # Ação - Homem-Aranha: Sem Volta para Casa
    """Danny Maccabee (Adam Sandler) queria um relacionamento sério, mas foi infeliz em sua
     tentativa de casamento. Para driblar a carência, passa a vivenciar somente namoricos
     e transas sem o menor compromisso. Assim, ele toca sua vida como cirurgião plástico bem
    sucedido, tendo sua melhor amiga Katherine (Jennifer Aniston), mãe solteira de um casal de pirralhos,
    como fiel escudeira. Mas um dia ele conhece a jovem Palmer (Brooklyn Decker) e a paixão toma conta de
    ambos. Disposto a se casar com ela, Danny pisa na bola quando, para conquistá-la, inventa que é marido
    da amiga, pai das crianças e que vai se separar. Começa então uma verdadeira aventura amorosa recheada
    de confusões de todos os tipos.""", # Comedia -  Esposa de Mentirinha




    """Invocação do Mal 4: O Último Ritual marca o desfecho da franquia de terror iniciada em
    2013 por James Wan. Os filmes são inspirados nas investigações sobrenaturais do famoso
    casal de paranormais norte-americanos Ed e Lorraine Warren, interpretados por Vera Farmiga e
    Patrick Wilson. Neste último capítulo, os Warren enfrentam mais um caso aterrorizante, desta vez
    envolvendo entidades misteriosas que desafiam sua experiência. Ed e Lorraine se veem obrigados
    a encarar seus maiores medos, colocando suas vidas em risco em uma batalha final contra forças malignas.
    O filme promete encerrar a história dos investigadores com suspense e momentos de tensão,
    consolidando a franquia como uma das mais populares do gênero. Além dos sustos, o longa também
    explora o relacionamento do casal, mostrando sua força emocional diante das adversidades.""", # Invocação do Mal - Horror/Terror

    """Após ver a Terra consumindo boa parte de suas reservas naturais, um grupo de
     astronautas recebe a missão de verificar possíveis planetas para receberem a população mundial,
    possibilitando a continuação da espécie. Cooper (Matthew McConaughey) é chamado para liderar o grupo
    e aceita a missão sabendo que pode nunca mais ver os filhos. Ao lado de Brand (Anne Hathaway),
    (Marlon Sanders) e Doyle (Wes Bentley), ele seguirá em busca de uma nova casa. Com o passar dos anos, sua filha Murph
     (Mackenzie Foy e Jessica Chastain) investirá numa própria jornada para também tentar salvar a população do planeta.""", # Interstellar - Sci-Fi

    """m Coringa, Arthur Fleck (Joaquin Phoenix) trabalha como palhaço para uma agência de talentos e, toda semana,
    precisa comparecer a uma agente social, devido aos seus conhecidos problemas mentais. Após ser demitido, Fleck
    reage mal à gozação de três homens em pleno metrô e os mata. Os assassinatos iniciam um movimento popular contra a elite
    de Gotham City, da qual Thomas Wayne (Brett Cullen) é seu maior representante.""" # Joker -  Drama
]

print("\nGenre Predictions for Example Synopses:")
for sinopse in example_synopses:
    predicted_genre, probabilities = predict_genre(sinopse)
    print(f"\nSinopse: {sinopse}")
    print(f"Gênero Previsto: {predicted_genre}")
    print("Probabilidades por Gênero:")
    for genre, prob in sorted(probabilities.items(), key=lambda item: item[1], reverse=True):
        print(f"- {genre}: {prob:.4f}")

## Resumo:

### Principais descobertas da análise de dados

* Foram inicialmente coletados 1000 filmes com os gêneros alvo ('Ação', 'Comédia', 'Drama', 'Ficção científica', 'Terror').
* O conjunto de dados foi balanceado para conter 786 filmes, com a seguinte distribuição entre os gêneros: 'Ação': 200, 'Drama': 200, 'Comédia': 200, 'Terror': 200 e 'Ficção científica': 200.
* O modelo retreinado alcançou uma acurácia de avaliação de aproximadamente 0,57 e um F1-score ponderado de cerca de 0,56 no conjunto de dados de teste.
* O modelo previu com sucesso o gênero para algumas sinopses de teste (por exemplo, uma sinopse de comédia prevista como 'Comédia', uma sinopse de ficção científica como 'Ficção científica').
* O modelo teve dificuldades com outras sinopses de teste, classificando incorretamente algumas (por exemplo, uma sinopse de ação/crime prevista como 'Terror', uma sinopse de drama prevista como 'Ação'), indicando dificuldade em distinguir entre certos gêneros.



### Exportar o modelo

Agora, vamos salvar o modelo treinado e o tokenizador para que você possa usá-los fora deste notebook.

Se usa esse modelo em ambientes python usando `AutoModelForSequenceClassification.from_pretrained(model_path)` e`AutoTokenizer.from_pretrained(tokenizer_path)`.

In [ ]:
# Save the model and tokenizer
model_path = "./movie_genre_model"
tokenizer_path = "./movie_genre_tokenizer"

trainer.save_model(model_path)
tokenizer.save_pretrained(tokenizer_path)

print(f"Model saved to {model_path}")
print(f"Tokenizer saved to {tokenizer_path}")

## Teste:

Foi realizado testes do modelo com prompts para cada genêro utilizado no treinamento

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


probs_series = pd.Series(probabilities)

# Sort probabilities in descending order
probs_series = probs_series.sort_values(ascending=False)

# Create a bar chart
plt.figure(figsize=(10, 6))
probs_series.plot(kind='bar')
plt.title('Probabilidade de Gênero para a Sinopse')
plt.xlabel('Gênero')
plt.ylabel('Probabilidade')
plt.ylim(0, 1) # Set y-axis limit from 0 to 1
plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for better readability
plt.tight_layout() # Adjust layout to prevent labels overlapping
plt.show()

In [ ]:
def grafico(probabilities):
  probs_series = pd.Series(probabilities)

  # Sort probabilities in descending order
  probs_series = probs_series.sort_values(ascending=False)

  # Create a bar chart
  plt.figure(figsize=(10, 6))
  probs_series.plot(kind='bar')
  plt.title('Probabilidade de Gênero para a Sinopse')
  plt.xlabel('Gênero')
  plt.ylabel('Probabilidade')
  plt.ylim(0, 1) # Set y-axis limit from 0 to 1
  plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for better readability
  plt.tight_layout() # Adjust layout to prevent labels overlapping
  plt.show()

In [ ]:
print("\nGenre Predictions for Example Synopses:")
for sinopse in example_synopses:
    predicted_genre, probabilities = predict_genre(sinopse)
    grafico(probabilities)
    print(f"\nSinopse: {sinopse}")
    print(f"Gênero Previsto: {predicted_genre}")
    print("Probabilidades por Gênero:")
    for genre, prob in sorted(probabilities.items(), key=lambda item: item[1], reverse=True):
        print(f"- {genre}: {prob:.4f}")

In [ ]:
def pie_chart_probabilities(probabilities):
    probs_series = pd.Series(probabilities)

    # Sort probabilities in descending order
    probs_series = probs_series.sort_values(ascending=False)

    # Create a pie chart
    plt.figure(figsize=(8, 8))
    plt.pie(probs_series, labels=probs_series.index, autopct='%1.1f%%', startangle=90)
    plt.title('Distribuição de Probabilidade de Gênero para a Sinopse')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

In [ ]:
print("\nGenre Predictions for Example Synopses (Pie Charts):")
for sinopse in example_synopses:
    predicted_genre, probabilities = predict_genre(sinopse)
    print(f"\nSinopse: {sinopse}")
    print(f"Gênero Previsto: {predicted_genre}")
    pie_chart_probabilities(probabilities)

In [ ]:
def predict_genre(sinopse):
    device = model.device
    inputs = tokenizer(sinopse, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).detach().cpu().numpy()[0]
    pred_id = probs.argmax()
    return le.inverse_transform([pred_id])[0], {genre: float(p) for genre, p in zip(le.classes_, probs)}

sinopse_teste = "Cientistas entram em outro planeta para estuda-lo"
predicted_genre, probabilities = predict_genre(sinopse_teste)

print(f"Sinopse: {sinopse_teste}")
print(f"Gênero Previsto: {predicted_genre}")
print("Probabilidades por Gênero:")
for genre, prob in sorted(probabilities.items(), key=lambda item: item[1], reverse=True):
    print(f"- {genre}: {prob:.4f}")

In [ ]:
transformer_eval_results = trainer.evaluate(test_dataset)
transformer_accuracy = transformer_eval_results['eval_accuracy']
transformer_f1 = transformer_eval_results['eval_f1']

print(f"Transformer - Accuracy: {transformer_accuracy:.4f}, F1-score: {transformer_f1:.4f}")